# Music Generation (Vocabulary Approach)

In [3]:
import torch

import numpy as np
import matplotlib.pyplot as plt

from IPython.core.debugger import set_trace
import h5py

%matplotlib inline

%load_ext autoreload
%autoreload 2

batch_size = 16
learning_rate = 1e-6
epochs = 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load training data

In [6]:
# Load X and y data for training
with h5py.File("../clean-data-1/data.h5", 'r') as hf:
    X = hf["X"][:]
    y = hf["y"][:]
    
print(X.shape) # Number of training Ex. x Length of sequence x 1 (following shape in)
print(y.shape) # Number of training Ex. x Vocab Size 
# y has been hot one encoded
# We might want to split thesse into train/val...

(9848, 50, 1)
(9848, 967)
